# PipeLine

**Instalasi Library**
*   menginstall beberapa library yang dibutuhkan



In [ ]:
!pip install mysqlclient
!pip install mysql-connector-python
!pip install pymysql
!pip install pyodbc
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysqlclient: filename=mysqlclient-2.1.1-cp38-cp38-linux_x86_64.whl size=109200 sha256=4a9c27898bca422f07ab5de00e8a759b2c643557c5450135061032d32d83cf08
  Stored in directory: /root/.cache/pip/wheels/5b/e1/84/a6185eaec318899f59a32d393af7729a0719cd93695d71f9a1
Successfully built mysqlclient
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 73.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 2.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━

**Mengambil data Iris dari SQL Server**
*   Mendefine engine yang nantinya akan terhubung dengan SQL Server
*   Melakukan select semua data pada tabel iris
*   Pada tabel di database ini terdapat kolom id dan petalwidth
*   Lalu menampilkan data menggunakan looping for



In [ ]:
import pyodbc 
import pandas as pd
cnxn_str = ("Driver={odbc driver 17 for sql server};"
            "Server=DESKTOP-JAFQH21\SQLEXPRESS;"
            "Database=prosain200411100186;"
            "Trusted_Connection=yes;")
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()	
cursor.execute("SELECT * FROM iris") 
row = cursor.fetchall() 
cursor.close()
data=[]
for i in row:
    data.append([i[0].strip('"'),i[1].strip('"'),i[2]])
df_sqlServer = pd.DataFrame(data, columns=['id','petalwidth','class'])
df_sqlServer


**Mengambil data Iris dari Elephant**
*   Mendefine engine yang nantinya akan terhubung dengan database di Elephant
*   Melakukan select semua data pada tabel iris
*   Pada tabel di database ini terdapat kolom id dan sepalwidth
*   Lalu menampilkan data menggunakan dataframe

In [ ]:
import psycopg2
import pandas as pd
# Connection parameters, yours will be different
param_dic = {
    "host"      : "floppy.db.elephantsql.com",
    "database"  : "qdbpvmbb",
    "user"      : "qdbpvmbb",
    "password"  : "qp3SRwpNs9sDtpF-e2xb4O913yIGPlCH"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

# Connect to the database
conn = connect(param_dic)

column_names = ['id','sepalwidth']

df_postEle = postgresql_to_dataframe(conn, "select * from iris", column_names)
df_postEle

**Mengambil data Iris dari PostgreSQL Lokal**
*   Mendefine engine yang nantinya akan terhubung dengan database PostgreSQL
*   Melakukan select semua data pada tabel iris
*   Pada tabel di database ini terdapat kolom id dan sepallength
*   Lalu menampilkan data menggunakan dataframe

In [ ]:
import psycopg2
import pandas as pd
# Connection parameters, yours will be different
param_dic = {
    "host"      : "localhost",
    "database"  : "prosain200411100186",
    "user"      : "postgres",
    "password"  : "123"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

# Connect to the database
conn = connect(param_dic)

column_names = ['id','sepallength']

df_postLocal = postgresql_to_dataframe(conn, "select * from iris", column_names)
df_postLocal

**Mengambil data Iris dari MySQL**
*   Mendefine engine yang nantinya akan terhubung dengan database di MySQL
*   Melakukan select semua data pada tabel iris
*   Pada tabel di database ini terdapat kolom id dan petallength
*   Lalu menampilkan data menggunakan dataframe

In [ ]:
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="prosain200411100186"
)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM iris")

myresult = mycursor.fetchall()
mycursor.close()
df_mysql = pd.DataFrame(myresult, columns=['id','petallength'])
# df_mysql.drop(df_mysql.columns[[0]],axis=1,inplace=True)
df_mysql

In [ ]:
# Menggabungkan data dari setiap database yg telah diakses sebelumnya
result = pd.concat([df_postLocal, df_postEle,df_mysql, df_sqlServer], axis=1)
# result = pd.merge(df_postLocal,df_sqlServer, how="inner", on=["id", "id"])
result

In [ ]:
#melakukan drop kolom id
result.drop(result.columns[[0,2,4,6]],axis=1,inplace=True)
result

In [ ]:
#memisah data untuk X dan Y
X = result.values[:, :4]
Y = result.values[:, 4]
print(X)

In [ ]:
#membagi data training dan data testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( 
    X, Y, test_size = 0.2, random_state = 100)

**Mengimplementasikan metode Desicion Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

In [ ]:
#mendefine variable y prediksi
y_pred = classifier.predict(X_test)

**Menghitung nilai akurasi**

In [ ]:
from sklearn.metrics import accuracy_score
print (" DecisionTree Accuracy : ",
    accuracy_score(y_test,y_pred)*100)

**Mengimplementasikan metode KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

In [ ]:
#mendefine variable y prediksi
y_pred = classifier.predict(X_test)

**Menghitung akurasi**

In [ ]:
print (" KNN Accuracy : ",
    accuracy_score(y_test,y_pred)*100)